In [2]:
import pandas as pd 

# 
# 1. Удаление дубликатов
# Для поиска и удаления дубликатов в данных используем метод drop_duplicates():

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes_cleaned = diabetes.drop_duplicates()
print(f"Осталось записей: {len(diabetes_cleaned)}")

Осталось записей: 768


2. Удаление неинформативных признаков
Неинформативные признаки — это те, где 95% значений повторяются или 95% значений уникальны. Проверим каждый признак:

In [3]:
non_informative_features = []
for column in diabetes_cleaned.columns:
    unique_ratio = diabetes_cleaned[column].nunique() / len(diabetes_cleaned)
    if unique_ratio > 0.95 or diabetes_cleaned[column].value_counts(normalize=True).max() > 0.95:
        non_informative_features.append(column)
print(non_informative_features)

['Gender']


3. Замена нулей на пропуски и доля пропусков в Insulin
Заменим нули на np.nan в указанных столбцах и найдем долю пропусков в столбце Insulin:

In [5]:
import numpy as np

columns_to_replace = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
diabetes_cleaned[columns_to_replace] = diabetes_cleaned[columns_to_replace].replace(0, np.nan)

missing_insulin = diabetes_cleaned['Insulin'].isnull().mean()
print(f"Доля пропусков в Insulin: {round(missing_insulin, 2)}")

Доля пропусков в Insulin: 0.49


C:\Users\Yoga\AppData\Local\Temp\ipykernel_53840\4094529712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_cleaned[columns_to_replace] = diabetes_cleaned[columns_to_replace].replace(0, np.nan)


4. Удаление признаков с более чем 30% пропусков
Удалим признаки, где пропусков больше 30%:

In [6]:
diabetes_cleaned = diabetes_cleaned.dropna(thresh=len(diabetes_cleaned) * 0.7, axis=1)
print(f"Осталось признаков: {len(diabetes_cleaned.columns)}")

Осталось признаков: 9


5. Удаление строк с более чем двумя пропусками
Удалим строки, где больше двух пропусков:

In [7]:
diabetes_cleaned = diabetes_cleaned.dropna(thresh=len(diabetes_cleaned.columns) - 2)
print(f"Осталось записей: {len(diabetes_cleaned)}")

Осталось записей: 761


6. Замена пропусков на медиану и среднее значение SkinThickness
Заменим пропуски на медиану и найдем среднее значение в столбце SkinThickness:

In [10]:
# Исключаем категориальные признаки (например, 'Gender') из обработки
numeric_columns = diabetes_cleaned.select_dtypes(include=[np.number]).columns

# Заменяем пропуски на медиану только для числовых столбцов
diabetes_cleaned[numeric_columns] = diabetes_cleaned[numeric_columns].fillna(diabetes_cleaned[numeric_columns].median())

# Находим среднее значение в столбце SkinThickness
mean_skin_thickness = diabetes_cleaned['SkinThickness'].mean()
print(f"Среднее значение SkinThickness: {round(mean_skin_thickness, 1)}")

Среднее значение SkinThickness: 29.1


7. Выбросы в SkinThickness методом межквартильного размаха
Найдем выбросы с помощью метода межквартильного размаха:

In [11]:
Q1 = diabetes_cleaned['SkinThickness'].quantile(0.25)
Q3 = diabetes_cleaned['SkinThickness'].quantile(0.75)
IQR = Q3 - Q1
outliers = diabetes_cleaned[(diabetes_cleaned['SkinThickness'] < (Q1 - 1.5 * IQR)) | (diabetes_cleaned['SkinThickness'] > (Q3 + 1.5 * IQR))]
print(f"Число выбросов: {len(outliers)}")

Число выбросов: 87


8. Выбросы в SkinThickness методом z-отклонения
Найдем выбросы с помощью метода z-отклонения:

In [14]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])

1


9. Выбросы в DiabetesPedigreeFunction методом межквартильного размаха
Найдем выбросы в DiabetesPedigreeFunction:

In [15]:
Q1 = diabetes_cleaned['DiabetesPedigreeFunction'].quantile(0.25)
Q3 = diabetes_cleaned['DiabetesPedigreeFunction'].quantile(0.75)
IQR = Q3 - Q1
outliers_dpf = diabetes_cleaned[(diabetes_cleaned['DiabetesPedigreeFunction'] < (Q1 - 1.5 * IQR)) | (diabetes_cleaned['DiabetesPedigreeFunction'] > (Q3 + 1.5 * IQR))]
print(f"Число выбросов: {len(outliers_dpf)}")

Число выбросов: 29
